<a href="https://colab.research.google.com/github/aravindk100/Learning-AI/blob/main/Intel_Gen_AI_Fine_tune_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
import datasets
from datasets import load_dataset


In [ ]:

#from sklearn.model_selection import train_test_split

# Load the IMDB dataset
imdb_dataset = load_dataset('imdb')
# Convert the dataset to lists for splitting
#x_data = dataset['train']['text'] + dataset['test']['text']
#y_data = dataset['train']['label'] + dataset['test']['label']

x_train = imdb_dataset['train']['text'][:100]
y_train = imdb_dataset['train']['label'][:100]
x_val = imdb_dataset['test']['text'][:10]
y_val = imdb_dataset['test']['label'][:10]

# Split the data into 90% training and 10% validation
#x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=42)

# Print the size of the splits
print(f'Training data size: {len(x_train)}, Training labels size: {len(y_train)}')
print(f'Validation data size: {len(x_val)}, Validation labels size: {len(y_val)}')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Training data size: 100, Training labels size: 100
Validation data size: 10, Validation labels size: 10


In [ ]:
x_val

['I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they hav

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset


# Prepare the data for the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the validation data
val_encodings = tokenizer(x_val, truncation=True, padding=True, max_length=512)

# Convert tokenized data to PyTorch tensors
val_input_ids = torch.tensor(val_encodings['input_ids'])
val_attention_mask = torch.tensor(val_encodings['attention_mask'])
val_labels = torch.tensor(y_val)

# Create a TensorDataset and DataLoader for validation data
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16)

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Evaluate the model
model.eval()  # Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        print(predictions, labels)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Validation accuracy without fine-tuning: {accuracy:.4f}')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1, 1, 1, 1, 1, 1, 0, 0, 1, 1]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Validation accuracy without fine-tuning: 0.2000


In [ ]:
!pip install transformers[torch]

In [ ]:
pip install accelerate -U

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Prepare the data for the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples, padding='max_length', truncation=True)

train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val, truncation=True, padding=True)

train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': y_train
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': y_val
})

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Function to compute additional metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {
        'accuracy': accuracy,
    }


# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print (eval_results)
print(f"Validation accuracy: {eval_results['eval_accuracy']}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.712100,0.642768,0.900000
2,0.620000,0.475608,1.000000
3,0.491800,0.328693,1.000000
4,0.275400,0.170011,1.000000
5,0.157800,0.061617,1.000000
6,0.074600,0.019276,1.000000
7,0.014900,0.008062,1.000000


In [ ]:
eval_results

{'eval_loss': 0.29541853070259094,
 'eval_runtime': 17.4305,
 'eval_samples_per_second': 0.574,
 'eval_steps_per_second': 0.057,
 'epoch': 3.0}